In [ ]:
from ALLCools.clustering import *
from wmb import *
import numpy as np
import pandas as pd
import anndata

import matplotlib.pyplot as plt
from ALLCools.plot import *

from ALLCools.integration.seurat_class import SeuratIntegration

In [ ]:
dataset = 'AIBS_SMART'
categorical_key = ['L1_annot', 'L2', 'L3', 'DissectionRegion']

## Input LSI before integration

In [ ]:
ref_adata = anndata.read_h5ad('mc_pca.h5ad')
query_adata = anndata.read_h5ad('rna_pca.h5ad')

In [ ]:
adata_list = [ref_adata, query_adata]

### Init empty adata_merge

In [ ]:
from scipy.sparse import csr_matrix

cells = sum([a.shape[0] for a in adata_list])
features = adata_list[0].shape[1]

adata_merge = anndata.AnnData(X=csr_matrix(([], ([], [])),
                                           shape=(cells, features)),
                              obs=pd.concat([a.obs for a in adata_list]),
                              var=adata_list[0].var)

In [ ]:
if dataset == 'AIBS_SMART':
    rna_annot = aibs.get_smart_annot()
elif dataset == 'AIBS_TENX':
    rna_annot = aibs.get_tenx_annot()
else:
    rna_annot = broad.get_tenx_annot()

mc_annot = cemba.get_mc_annot()

In [ ]:
rna_annot

In [ ]:
for key in categorical_key:
    adata_merge.obs[key] = pd.concat(
        [mc_annot[key].to_pandas(), 
         rna_annot[key].to_pandas()]
    )

In [ ]:
for adata in adata_list:
    for key in categorical_key:
        adata.obs[key] = adata_merge.obs[key]

In [ ]:
adata_list

In [ ]:
adata_merge

## Integration and transform

In [ ]:
integrator = SeuratIntegration()

In [ ]:
anchor = integrator.find_anchor(adata_list,
                                k_local=None,
                                key_local='X_pca',
                                k_anchor=5,
                                key_anchor='X',
                                dim_red='cca',
                                max_cc_cells=100000,
                                k_score=30,
                                k_filter=None,
                                scale1=False,
                                scale2=False,
                                n_components=50,
                                n_features=200,
                                alignments=[[[0], [1]]])

In [ ]:
corrected = integrator.integrate(key_correct='X_pca',
                                 row_normalize=True,
                                 n_components=30,
                                 k_weight=100,
                                 sd=1,
                                 alignments=[[[0], [1]]])

adata_merge.obsm['X_pca_integrate'] = np.concatenate(corrected)

## Label transfer

In [ ]:
transfer_results = integrator.label_transfer(
    ref=[0],
    qry=[1],
    categorical_key=categorical_key,
    key_dist='X_pca'
)

In [ ]:
for k, v in transfer_results.items():
    v.to_hdf(f'{k}_transfer.hdf', key='data')

In [ ]:
integrator.save_transfer_results_to_adata(adata_merge, transfer_results)

## Save

In [ ]:
adata_merge.write_h5ad('final.h5ad')

In [ ]:
adata_merge

In [ ]:
integrator.save('integration')